In [1]:
import torch as th
from torch.nn.modules.loss import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader, TensorDataset
import my
import resnet

In [2]:
train_x, train_y, test_x, test_y = my.load_cifar10(rbg=True, torch=True)

train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=4096)

n_classes = int(train_y.max() - train_y.min() + 1)

In [3]:
c = resnet.ResNet(18, n_classes)
cuda = True # always GPU 0
if cuda:
    c.cuda()
optim = SGD(c.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)

In [4]:
n_epochs = 1
for i in range(n_epochs):
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        ce = CrossEntropyLoss()(c(x), y)
        optim.zero_grad()
        ce.backward()
        optim.step()
        
    accuracy = my.parallel_global_stats(c, test_loader, my.accuracy, range(4))
    print('[epoch %d]accuracy: %f' % ((i + 1), float(accuracy)))

[epoch 1]accuracy: 0.580000


In [5]:
# for pg in optim.param_groups:
#     pg['lr'] *= 0.1

In [6]:
# th.save(c.state_dict(), 'cifar10-resnet18')
# th.save(c.state_dict(), 'cifar10-1-9-resnet18')